Let's import some basic required packages.

In [4]:
!pip install xgboost 

In [1]:
#  Import some data manipulation and plotting packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error,median_absolute_error,mean_squared_log_error, r2_score
from sklearn.model_selection import GridSearchCV

# Read Datasets

In [2]:
# Read all the given datasets into separate dataframes
aquifers = pd.read_csv('C:/Users/hp/Desktop/Acea-smart-water-analytics-main/Feature Engineered Datasets/aquifers_fe.csv')
lakes = pd.read_csv('C:/Users/hp/Desktop/Acea-smart-water-analytics-main/Feature Engineered Datasets/lakes_fe.csv')
rivers = pd.read_csv('C:/Users/hp/Desktop/Acea-smart-water-analytics-main/Feature Engineered Datasets/rivers_fe.csv')
springs = pd.read_csv('C:/Users/hp/Desktop/Acea-smart-water-analytics-main/Feature Engineered Datasets/springs_fe.csv')

# Model

## Aquifers

In [3]:
aquifers.head()

,Mean_Rainfall,Mean_Temp,Actual_Depth,Actual_Volume,Actual_Hydrometry,Date
0,0.415556,6.6250,-6.08026,-8019.271158,-0.083056,1998-01-04
1,2.054444,6.0750,-6.06452,-7956.571285,-0.104167,1998-01-05
2,0.921111,9.0875,-6.15706,-7715.808854,0.011944,1998-01-06
3,0.878889,12.3250,-6.10774,-7731.378766,-0.008611,1998-01-07
4,0.908889,12.6500,-6.05310,-7812.676449,-0.072222,1998-01-08


In [4]:
# Normalizing the columns to remove negative values
aquifers['Mean_Rainfall'] = (aquifers['Mean_Rainfall'] - aquifers['Mean_Rainfall'].min()) / (aquifers['Mean_Rainfall'].max() - aquifers['Mean_Rainfall'].min())
aquifers['Mean_Temp'] = (aquifers['Mean_Temp'] - aquifers['Mean_Temp'].min()) / (aquifers['Mean_Temp'].max() - aquifers['Mean_Temp'].min())
aquifers['Actual_Depth'] = (aquifers['Actual_Depth'] - aquifers['Actual_Depth'].min()) / (aquifers['Actual_Depth'].max() - aquifers['Actual_Depth'].min())
aquifers['Actual_Volume'] = (aquifers['Actual_Volume'] - aquifers['Actual_Volume'].min()) / (aquifers['Actual_Volume'].max() - aquifers['Actual_Volume'].min())
aquifers['Actual_Hydrometry'] = (aquifers['Actual_Hydrometry'] - aquifers['Actual_Hydrometry'].min()) / (aquifers['Actual_Hydrometry'].max() - aquifers['Actual_Hydrometry'].min())

In [5]:
aquifers.head()

,Mean_Rainfall,Mean_Temp,Actual_Depth,Actual_Volume,Actual_Hydrometry,Date
0,0.007267,0.295126,0.984879,0.395764,0.090032,1998-01-04
1,0.035926,0.276216,0.985372,0.401094,0.084072,1998-01-05
2,0.016108,0.379792,0.982474,0.421561,0.116854,1998-01-06
3,0.015369,0.491104,0.984019,0.420238,0.111050,1998-01-07
4,0.015894,0.502278,0.985729,0.413327,0.093091,1998-01-08


In [9]:
# separating the date and target feature
X = aquifers.drop(['Date','Actual_Depth'],axis=1)
y = aquifers['Actual_Depth']

In [10]:
#Dividing the dataset into train and test for features as well as labels
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
#checking the number of rows in training data
len(X_train)

6523

In [12]:

#checking the number of rows in training labels
len(y_train)

6523

In [13]:
#checking the number of rows in testing data
len(X_test)

1631

In [14]:
#checking the number of rows in testing labels
len(y_test)

1631

### knn

In [15]:
# create a knn regressor
neigh = KNeighborsRegressor()

In [16]:
# find optimal parameters with grid search
param_grid = {
    'n_neighbors': [3,5,7,9,11,19],
    'weights': ['uniform', 'distance'],
    'metric' : ['euclidean','manhattan']
}
grid = GridSearchCV(estimator=neigh, param_grid=param_grid, cv=10,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [17]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


In [18]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.0028144705118891005
Best Params:  {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


In [19]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [20]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [21]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.02103068051342738

In [22]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.05328383316662198

In [23]:
# find r2 score
r2_score(y_test,y_pred)

0.9607730924181563

### Multiple Linear Regression

In [24]:
# create a linear regressor
lr = LinearRegression()

In [25]:
# fit the lr on the train data
lr = lr.fit(X_train, y_train)

In [26]:
# find predictions for test data
y_pred = lr.predict(X_test)

In [27]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [28]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.3866462761518887

In [29]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.31056503793039225

In [30]:
# find r2 score
r2_score(y_test,y_pred)

-0.361099450434454

R2  is negative only when the chosen model does not follow the trend of the data, so fits worse than a horizontal line.

### Random Forest

In [33]:
# create a random forest regressor
rf = RandomForestRegressor()

In [34]:
# find optimal parameters with grid search
param_grid = {
'bootstrap': [True, False],
 'max_depth': [None,2,10, 20, 50, 100],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100,200,500, 1000]
             }
grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.0021697543201036028
Best Params:  {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.019655865584285948

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.04842439087861878

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.9668303549718164

### SGRegression

In [ ]:
# create a linear regressor
lr = SGDRegressor()

In [ ]:
# find optimal parameters with grid search
param_grid = {
    
    'loss' : ['squared_loss','huber'],
    'alpha': [0.0001,0.001,0.01,0.1],
     'eta0': [0.01,0.1,1,10,100],
    'tol' : [0.00001,0.0001,0.001,0.01,0.1]    
 }
grid = GridSearchCV(estimator=lr, param_grid=param_grid,cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 202 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   25.9s finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.3227038141370854

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.2877640670063738

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

-0.1718414332285927

### Decision Tree Regressor

In [ ]:
dt = DecisionTreeRegressor()

In [ ]:
# find optimal parameters with grid search
param_grid = {
    
    'criterion' : ['mse','mae'],
    'max_depth': range(10,20),
     'min_samples_split': range(2,10),
    'min_samples_leaf' : range(1,5)    
 }
grid = GridSearchCV(estimator=dt, param_grid=param_grid, cv=10,
                    scoring=['neg_median_absolute_error','neg_mean_squared_log_error','r2'], refit = 'neg_mean_squared_log_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 10 folds for each of 640 candidates, totalling 6400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 40.9min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 64.8min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 90.8min
[Parallel(n_jobs=-1)]: Done 6400 out of 6400 | elapsed: 99.5min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.0029725895160034803
Best Params:  {'criterion': 'mae', 'max_depth': 12, 'min_samples_leaf': 2, 'min_samples_split': 4}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.01684240996782499

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.05367391890221074

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.9561599730076912

### XGBoost Regressor

In [ ]:
xgbtree = XGBRegressor()

In [ ]:
# find optimal parameters with grid search
param_grid = {
              'learning_rate': [0.001,0.01,0.1,1],
              'max_depth': range(2,10),
              'subsample':[0.01,0.1,1] ,
              'colsample_bytree': [0.01,0.1,1],
              'n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=xgbtree, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 19.9min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 27.1min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 35.3min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 45.0min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 60.7min
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed: 74.4min finished


[12:42:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.016081142855098807
Best Params:  {'colsample_bytree': 1, 'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 1000, 'subsample': 1}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.02528232677151125

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.053061569375817304

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.9585200503118089

### AdaBoost Regressor

In [ ]:
abreg = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(),
                              random_state=0)

In [ ]:
# find optimal parameters with grid search
param_grid = { 'loss' : ['linear', 'square', 'exponential'],
              'learning_rate': [0.001,0.01,0.1,1],
              'base_estimator__max_depth': range(2,10),
              'n_estimators': [100,200,500,1000]}
grid = GridSearchCV(estimator=abreg, param_grid=param_grid, cv=5,
                    scoring=['neg_median_absolute_error','r2'], refit = 'neg_median_absolute_error',
                    verbose=1, n_jobs=-1)

In [ ]:
# fit the grid on the train data
grid_result = grid.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 53.7min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 100.0min
[Parallel(n_jobs=-1)]: Done 1920 out of 1920 | elapsed: 108.9min finished


In [ ]:
# print the best score and parameters found
print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Best Score:  -0.014184367982375096
Best Params:  {'base_estimator__max_depth': 9, 'learning_rate': 1, 'loss': 'square', 'n_estimators': 200}


In [ ]:
# find predictions for test data
y_pred = grid_result.best_estimator_.predict(X_test)

In [ ]:
#normalizing y_pred
y_pred = (y_pred - y_pred.min())/(y_pred.max() - y_pred.min())

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.02192634636241439

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.051313886313701484

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.9641048020090041

### MLP Regressor

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [ ]:
# create a directory to save the model weights
import os
os.mkdir('model_aquifer_save')

In [ ]:
'''Callbacks'''
#file path, it saves the model in the 'model_aquifers_save' folder
#and we are monitoring model with val_root_mean_squared_error
filepath="model_aquifer_save/weights.h5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_root_mean_squared_error', verbose=1, save_best_only=True, mode='auto')

In [ ]:
import datetime

In [ ]:
# the log directory path is used to write the tensorboard logs
log_dir="logs_aquifer\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

In [ ]:
# Creating a Neural Network Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Dense(1000,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dense(1000,activation='relu'))
model.add(Dense(1))

In [ ]:
import tensorflow.keras.backend as K

In [ ]:

def rmsle(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

In [ ]:
model.compile(optimizer='Adam',loss=rmsle,metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          callbacks=[checkpoint,tensorboard_callback],
          batch_size=128,epochs=100)


Epoch 1/100
51/51 [==============================] - 5s 77ms/step - loss: 0.2173 - root_mean_squared_error: 0.3251 - val_loss: 0.1129 - val_root_mean_squared_error: 0.1807

Epoch 00001: val_root_mean_squared_error improved from inf to 0.18073, saving model to model_aquifer_save/weights.h5
Epoch 2/100
51/51 [==============================] - 3s 67ms/step - loss: 0.0996 - root_mean_squared_error: 0.1531 - val_loss: 0.1082 - val_root_mean_squared_error: 0.1678

Epoch 00002: val_root_mean_squared_error improved from 0.18073 to 0.16782, saving model to model_aquifer_save/weights.h5
Epoch 3/100
51/51 [==============================] - 3s 68ms/step - loss: 0.0849 - root_mean_squared_error: 0.1313 - val_loss: 0.0783 - val_root_mean_squared_error: 0.1181

Epoch 00003: val_root_mean_squared_error improved from 0.16782 to 0.11809, saving model to model_aquifer_save/weights.h5
Epoch 4/100
51/51 [==============================] - 3s 67ms/step - loss: 0.0758 - root_mean_squared_error: 0.1164 - val_l

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              5000      
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 1001      
Total params: 3,009,001
Trainable params: 3,009,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# find median absolute error
median_absolute_error(y_test,y_pred)

0.020680591275962612

In [ ]:
# find root mean square log error
np.sqrt(mean_squared_log_error(y_test,y_pred))

0.0506280141315964

In [ ]:
# find r2 score
r2_score(y_test,y_pred)

0.9561026133850481